In [ ]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [ ]:
title = "data analyst"
location = "Australia"
start = 0
all_jobs = []

In [ ]:
while True:
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&start={start}"

    # Send request
    response = requests.get(list_url, headers={"User-Agent": "Mozilla/5.0"})

    # Parse the response
    list_soup = BeautifulSoup(response.text, "html.parser")
    page_jobs = list_soup.find_all("li")

    # Print start index and jobs found on this page
    print(f"Start index: {start}, Jobs found: {len(page_jobs)}")

    if len(page_jobs) == 0:
        print("No more jobs found, scraping finished.")
        break

    all_jobs.extend(page_jobs)
    start += 25  # move to next page


Start index: 0, Jobs found: 10
Start index: 25, Jobs found: 10
Start index: 50, Jobs found: 10
Start index: 75, Jobs found: 10
Start index: 100, Jobs found: 10
Start index: 125, Jobs found: 10
Start index: 150, Jobs found: 10
Start index: 175, Jobs found: 10
Start index: 200, Jobs found: 10
Start index: 225, Jobs found: 10
Start index: 250, Jobs found: 10
Start index: 275, Jobs found: 10
Start index: 300, Jobs found: 10
Start index: 325, Jobs found: 10
Start index: 350, Jobs found: 10
Start index: 375, Jobs found: 10
Start index: 400, Jobs found: 10
Start index: 425, Jobs found: 10
Start index: 450, Jobs found: 10
Start index: 475, Jobs found: 10
Start index: 500, Jobs found: 10
Start index: 525, Jobs found: 10
Start index: 550, Jobs found: 10
Start index: 575, Jobs found: 10
Start index: 600, Jobs found: 10
Start index: 625, Jobs found: 10
Start index: 650, Jobs found: 10
Start index: 675, Jobs found: 10
Start index: 700, Jobs found: 10
Start index: 725, Jobs found: 10
Start index: 75

In [ ]:
id_list = []
skill_keywords = ["Python", "SQL", "Excel", "Power BI", "Tableau", "R", "Machine Learning",
                  "Communication", "Data Visualization", "Statistics", "ETL", "Azure", "AWS", "Google Cloud"]

In [ ]:
id_list = []

for job in all_jobs:
    base_card_div = job.find("div", {"class": "base-card"})
    if base_card_div and base_card_div.get("data-entity-urn"):
        job_id = base_card_div.get("data-entity-urn").split(":")[3]
        print(job_id)
        id_list.append(job_id)


4291080502
4278720962
4279997293
4281532130
4289813268
4291560097
4278792395
4288516909
4281151968
4220599987
4288724158
4291228115
4282105076
4262326983
4288155263
4281542130
4287118450
4291554535
4289749722
4287360242
4290567120
4220902567
4283860911
4281302866
4290004964
4291348352
4288318158
4220903441
4283492035
4273297800
4271764946
4290674351
4264320839
4288724154
4283476842
4274601885
4285780885
4280147164
4290671618
4286253993
4280011394
4257669659
4280750786
4273539019
4288370326
4255687707
4285225710
4283161481
4227270377
4246251556
4287334691
4278733643
4157816699
4291304315
4288723679
4291264713
4279274991
4278070410
4279275849
4291262727
4266025068
4279274992
4241856077
4282728875
3778387011
4290814843
4286907781
4287925481
4284424569
4240286535
4146526265
4289203008
4289452049
4291012508
4284044906
4287843932
4288655919
4290532684
4279953728
4288744007
4273383270
4288722789
4287026772
4286634703
4285817312
4288859388
4278750498
4289443052
4286758592
4263357183
4287313178

In [ ]:
job_list = []
import re

# Loop through the list of job IDs and get each URL
for job_id in id_list:
    # Construct the URL for each job using the job ID
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"

    # Send a GET request to the job URL and parse the reponse
    job_response = requests.get(job_url)
    print(job_response.status_code)
    if job_response.status_code != 200:
        break
    job_soup = BeautifulSoup(job_response.text, "html.parser")

     # Create a dictionary to store job details
    job_post = {}

    # Try to extract and store the job title
    try:
        job_post["job_title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
    except:
        job_post["job_title"] = None


    try:
        job_post["Location"] = job_soup.find("span", {"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job_post["Location"] = None


    try:
        skills_section = job_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"})
        skills_list = []

        if skills_section:
            desc_text = skills_section.get_text(" ", strip=True)

            # Predefined list of common skills
            skill_keywords = [
                "Python", "R", "SQL", "Excel", "Power BI", "Tableau",
                "Machine Learning", "Statistics", "ETL", "AWS", "Azure", "Snowflake",
                "Communication", "Problem-solving", "Data Visualization"
            ]

            # Search for skills inside description
            for skill in skill_keywords:
                if re.search(rf"\b{skill}\b", desc_text, re.IGNORECASE):
                    skills_list.append(skill)

        job_post["skills"] = skills_list if skills_list else None
    except:
        job_post["skills"] = None


    try:
    # Get job description text
        job_desc_tag = job_soup.find("div", {"class": "show-more-less-html__markup"})
        job_desc = job_desc_tag.get_text(" ", strip=True) if job_desc_tag else ""

        # Extract salary using regex
        salary_match = re.search(
            r"(\$[0-9]{2,3}[,0-9]*\s*(?:-\s*\$[0-9]{2,3}[,0-9]*)?\s*(?:k|K)?|AUD\s*[0-9,]+(?:\s*-\s*AUD\s*[0-9,]+)?)",
            job_desc,
            re.IGNORECASE
        )

        if salary_match:
            job_post["salary"] = salary_match.group(0)
        elif "competitive salary" in job_desc.lower():
            job_post["salary"] = "Competitive salary"
        else:
            job_post["salary"] = None

    except Exception as e:
        job_post["salary"] = None


    # Try to extract and store the company name
    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
    except:
        job_post["company_name"] = None

    # # Try to extract and store the time posted
    # try:
    #     job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    # except:
    #     job_post["time_posted"] = None

    # # Try to extract and store the number of applicants
    # try:
    #     job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
    # except:
    #     job_post["num_applicants"] = None


    # Append the job details to the job_list
    job_list.append(job_post)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
429


In [ ]:
job_list

[{'job_title': 'Senior Data Analyst',
  'Location': 'North Sydney, New South Wales, Australia',
  'skills': ['Python', 'SQL', 'Excel', 'Power BI', 'Tableau', 'Communication'],
  'salary': None,
  'company_name': 'NielsenIQ'},
 {'job_title': 'Data Analyst',
  'Location': 'Melbourne, Victoria, Australia',
  'skills': ['Python', 'R', 'SQL', 'Power BI', 'Tableau', 'Problem-solving'],
  'salary': None,
  'company_name': 'carsales'},
 {'job_title': 'Data Analyst',
  'Location': 'Sydney, New South Wales, Australia',
  'skills': ['Python', 'R', 'SQL', 'Power BI', 'Tableau', 'Communication'],
  'salary': '$500 ',
  'company_name': 'Hays'},
 {'job_title': 'Product Data Analyst',
  'Location': 'Sydney, New South Wales, Australia',
  'skills': ['Tableau'],
  'salary': None,
  'company_name': 'Tata Consultancy Services'},
 {'job_title': 'Junior Data Operations Analyst - Hybrid',
  'Location': 'Southbank, Victoria, Australia',
  'skills': ['Python', 'SQL', 'Excel', 'Problem-solving'],
  'salary': 'a

In [ ]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
df = pd.DataFrame(job_list)
df

,job_title,Location,skills,salary,company_name
0,Senior Data Analyst,"North Sydney, New South Wales, Australia","[Python, SQL, Excel, Power BI, Tableau, Commun...",None,NielsenIQ
1,Data Analyst,"Melbourne, Victoria, Australia","[Python, R, SQL, Power BI, Tableau, Problem-so...",None,carsales
2,Data Analyst,"Sydney, New South Wales, Australia","[Python, R, SQL, Power BI, Tableau, Communicat...",$500,Hays
3,Product Data Analyst,"Sydney, New South Wales, Australia",[Tableau],None,Tata Consultancy Services
4,Junior Data Operations Analyst - Hybrid,"Southbank, Victoria, Australia","[Python, SQL, Excel, Problem-solving]","aud,",Experian Asia Pacific
5,Data Analyst,"New South Wales, Australia","[Python, SQL, Excel]","$110,000 - $130,000",Searchability®
6,Data Analyst,"Canberra, Australian Capital Territory, Australia","[Python, R, SQL, Excel, Power BI, Problem-solv...",$99k,Indigeco Pty Ltd
7,Data Analyst,"Sydney, New South Wales, Australia","[Python, R, Excel, Power BI, Communication]","$90,000",Aurora Education Foundation
8,Data Analyst,"Perth, Western Australia, Australia","[Python, R, SQL, Power BI, Machine Learning, S...",$214,AFG
9,Data Analyst,Australia,[Statistics],$40,DataAnnotation


In [ ]:
#Save data to CSV file
df.to_csv('Australian_Job_Market.csv', index=False)
from google.colab import files
files.download('Australian_Job_Market.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(df.shape)       # number of rows & columns
print(df.columns)     # column names
print(df.head())      # first 5 rows
print(df.info())      # data types & missing values

(399, 5)
Index(['job_title', 'Location', 'skills', 'salary', 'company_name'], dtype='object')
                                 job_title  \
0                      Senior Data Analyst   
1                             Data Analyst   
2                             Data Analyst   
3                     Product Data Analyst   
4  Junior Data Operations Analyst - Hybrid   

                                   Location  \
0  North Sydney, New South Wales, Australia   
1            Melbourne, Victoria, Australia   
2        Sydney, New South Wales, Australia   
3        Sydney, New South Wales, Australia   
4            Southbank, Victoria, Australia   

                                              skills salary  \
0  [Python, SQL, Excel, Power BI, Tableau, Commun...   None   
1  [Python, R, SQL, Power BI, Tableau, Problem-so...   None   
2  [Python, R, SQL, Power BI, Tableau, Communicat...  $500    
3                                          [Tableau]   None   
4              [Python, SQL, Ex

In [ ]:
total_rows = df.shape[0]
print(f"Total rows: {total_rows}")

# Method 2: Using len()
total_rows = len(df)
print(f"Total rows: {total_rows}")

Total rows: 399
Total rows: 399
